In [1]:
import time
from olsEmpowered import sim_data         as sd
from olsEmpowered import power_estimation as pe
from olsEmpowered import isotonic         as iso
from olsEmpowered import binary_search    as bst

/home/bknight/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [ ]:
dgps_analyzed = 0

while dgps_analyzed < 2:
    start = time.time()
    dgp = sd.create_random_dgp(max_covariates             = 10, 
                               permissible_distributions  = ['normal', 
                                                             'exponential', 
                                                             'uniform'],
                               range_of_normal_loc        = [50, 250], 
                               range_of_normal_scale      = [5, 25],
                               range_of_exponential_scale = [0.5, 4],
                               range_of_uniform           = [0,10],
                               range_of_betas             = [0, 2.5],
                               range_of_abs_mde           = [0.10, 0.20],
                               range_of_noise_loc         = [25.0, 50.0],
                               range_of_noise_scale       = [20.00, 35.00],
                               sample_size                = 4500000)
    pe_ob = pe.power_estimation(dgp)
    print("############## Starting value is n = {:,}.    ###########".format(pe_ob.starting_value))
    print("############## Starting adj. R-squared is {}. ###########".format(round(pe_ob.rsquared_adj,2)))
    if pe_ob.starting_value > 475000: 
        sd.remove_data(dgp, drop_meta_data = True)
        print("Starting value too high.")
        continue
    if pe_ob.starting_value < 500: 
        sd.remove_data(dgp, drop_meta_data = True)
        print("Starting value too low.")
        continue       
    if pe_ob.rsquared_adj > 0.95:
        sd.remove_data(dgp, drop_meta_data = True)
        print("r-squared too high.")
        continue 
    
    iso_ob = iso.isotonic(pe_ob)
    n1, p1,  df1 = iso_ob.isotonic_interpolation()
   
    naive_bst = bst.binary_search(pe_ob, informed = 0)
    n2, p2,  df2 = naive_bst.binary_search()

    informed_bst = bst.binary_search(pe_ob, informed = 1)
    n3, p3,  df3 = informed_bst.binary_search()
    
    pe.save_results(df1, iso_ob)
    del iso_ob
    pe.save_results(df2, naive_bst)
    del naive_bst
    pe.save_results(df3, informed_bst)
    del informed_bst

    dgps_analyzed += 1
    sd.remove_data(dgp, drop_meta_data = False)
    end = time.time()
    time_elapsed = end - start
    time_min = int(time_elapsed/60)
    time_sec = int(time_elapsed - (time_min*60))
    print("Assessment took {} minutes and {} seconds.".format(time_min, time_sec))    

7 additional covariates will be created.


/home/bknight/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2223: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Now performing OLS to infer DGP parameters.
Simulation data was saved to: 
     /home/bknight/Documents/Power_Analysis_Techniques/v3/data/sim_data_2020_01_04_163104.csv.
Meta-data was saved to: 
     /home/bknight/Documents/Power_Analysis_Techniques/v3/data/log_files/sim_data_2020_01_04_163104_log_file.txt.
4500000 observations of simulation data created in 1 minutes and 54 seconds.
Reconstituting data object from file.
Successfully read in the .csv file specified at:
     /home/bknight/Documents/Power_Analysis_Techniques/v3/data/sim_data_2020_01_04_163104.csv.
/home/bknight/Documents/Power_Analysis_Techniques/v3/data/log_files/sim_data_2020_01_04_163104_log_file.txt
Successfully read in the meta-data specified at:
     /home/bknight/Documents/Power_Analysis_Techniques/v3/data/log_files/sim_data_2020_01_04_163104_log_file.txt.
############## Starting value is n = 525,308.    ###########
############## Starting adj. R-squared is 0.81. ###########
/home/bknight/Documents/Power_Analysis_T

/home/bknight/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2223: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


The effective power of sample size n = 1,355,012 is 100%.
Estimating the effective power of n = 1,120,535 using 200 simulations.


/home/bknight/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2223: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


The effective power of sample size n = 1,120,535 is 96.52%.
Estimating the effective power of n = 804,699 using 200 simulations.


/home/bknight/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2223: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


The effective power of sample size n = 804,699 is 83.58%.
Estimating the effective power of n = 747,163 using 200 simulations.


/home/bknight/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2223: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [ ]:
import pandas as pd
import os, os.path

files_found = len([name for name in os.listdir('./results/')])
print("{} files were found in total.".format(files_found))
print("{} iterations.".format(int(files_found/3)))
isotonic_results        = []
naive_binary_results    = []
informed_binary_results = []
[isotonic_results.append(name) for name in os.listdir('./results/') if "isotonic" in name]
[naive_binary_results.append(name) for name in os.listdir('./results/') if "naive_binary" in name]
[informed_binary_results.append(name) for name in os.listdir('./results/') if "informed_binary" in name]
isotonic_results.sort()
naive_binary_results.sort()
informed_binary_results.sort()


iso_sims, iso_time, iso_n = [], [], []
for i in isotonic_results:
    df = pd.read_csv('./results/' + i)
    df['delta'] = abs(df['power'] - 0.8)
    df.sort_values('delta',ascending=True, inplace=True)
    n = df.iat[0,0]
    iso_sims.append(df['sims_used'][0])
    iso_time.append(df['seconds_used'][0])
    iso_n.append(n)
   # print(df)
    
naive_sims, naive_time, naive_n = [], [], []
for j in naive_binary_results:
    df = pd.read_csv('./results/' + j)
    df['delta'] = abs(df['power'] - 0.8)
    df.sort_values('delta',ascending=True, inplace=True)
    n = df.iat[0,0]
    naive_sims.append(df['sims_used'][0])
    naive_time.append(df['seconds_used'][0])
    naive_n.append(n)
   # print(df)
    
informed_sims, informed_time, informed_n = [], [], []
for k in informed_binary_results:
    df = pd.read_csv('./results/' + k)
    df['delta'] = abs(df['power'] - 0.8)
    df.sort_values('delta',ascending=True, inplace=True)
    n = df.iat[0,0]
    informed_sims.append(df['sims_used'][0])
    informed_time.append(df['seconds_used'][0])
    informed_n.append(n)
#     print(df)

In [ ]:
results_df = pd.DataFrame([iso_sims, iso_time, iso_n, 
                           naive_sims, naive_time, naive_n, 
                           informed_sims, informed_time, informed_n]).T
results_df.columns = ['iso_sims', 'iso_time', 'iso_n', 
                      'naive_sims', 'naive_time', 'naive_n', 
                      'informed_sims', 'informed_time', 'informed_n']
results_df.sort_values(by=['iso_n'], ascending=False)

In [ ]:
subset = results_df[ (results_df['iso_n'] <= 1000000)
                   & (results_df['informed_n'] <= 1000000)
                   & (results_df['naive_n'] <= 1000000)]
subset.describe()

In [ ]:
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


fig, ax = plt.subplots()
fig.set_size_inches(12, 4.8, forward=True)



x1 = subset['iso_n']
y1 = subset['iso_time']


x2 = subset['informed_n']
y2 = subset['informed_time']

x3 = subset['naive_n'] 
y3 = subset['naive_time']

sns.regplot(x1, y1, lowess=True, 
            color = '#b300b3', 
            marker='o', scatter_kws={'s':10})
sns.regplot(x2, y2, lowess=True, 
            color = '#006600', 
            marker='o', scatter_kws={'s':10})
sns.regplot(x3, y3, lowess=True, 
            color = '#3333cc', 
            marker='o', scatter_kws={'s':10})

labels = [
         'Isotonic Interpolation', 
         'Informed Binary Search', 
         'Naive Binary Search'
         ]
plt.legend(labels, prop={'size': 14})
plt.ylabel('Estimation \nTime \nin \nSeconds', size = 18).set_rotation(0)
plt.xlabel('Estimated Sample Size Required', size = 18)
plt.xlim(0,1000000)

ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,}'.format(int(x))))

ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,}'.format(int(x))))

ax.tick_params(axis='both', which='major', labelsize=16)
ax.yaxis.set_label_coords(-0.16,0.35)

plt.show()
# plt.savefig('v1.png',bbox_inches='tight')
plt.clf()